In [1]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# LeNet5 - CALTECH101

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.lenet5_network import LeNet5Network

Using TensorFlow backend.


In [3]:
network_name = 'lenet5'
fit_epochs = 50
dataset = 'caltech101'
dataset_percentage = '40'

In [4]:
network = LeNet5Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for caltech101_40...
Found 2451 images belonging to 101 classes.
Loading test set for caltech101_40...
Found 735 images belonging to 101 classes.
Loading validation set for caltech101_40...
Found 398 images belonging to 101 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [8]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
77/77 [==============================] - 5s 60ms/step - loss: 6.0448 - accuracy: 0.1142 - precision: 0.0779 - recall: 0.0286 - val_loss: 3.7770 - val_accuracy: 0.1973 - val_precision: 0.1739 - val_recall: 0.0754

Epoch 00001: val_loss improved from inf to 3.77702, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/caltech101/best.h5
Epoch 2/50
77/77 [==============================] - 3s 41ms/step - loss: 3.7852 - accuracy: 0.2297 - precision: 0.5703 - recall: 0.4394 - val_loss: 3.4989 - val_accuracy: 0.2395 - val_precision: 0.7609 - val_recall: 0.7217

Epoch 00002: val_loss improved from 3.77702 to 3.49894, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/caltech101/best.h5
Epoch 3/50
77/77 [==============================] - 4s 54ms/step - loss: 3.4696 - accuracy: 0.2636 - precision: 0.6684 - recall: 0.6110 - val_loss: 4.0677 - val_accuracy: 0.2612 - val_precision: 0.5652 - val_recall: 0.5254

Epoch 00003: val_loss did no

In [9]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## Results

### Epoch CSV

In [10]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

epoch, time, val_loss, val_accuracy, val_precision, val_recall
0,0,0,0,0,0
1,4.684413194656372,3.7770187854766846,0.1972789168357849,0.17391301691532135,0.07536231726408005
2,3.1751291751861572,3.4989421367645264,0.23945578932762146,0.760869562625885,0.7217391729354858
3,4.177363872528076,4.0677313804626465,0.2612244784832001,0.5652173757553101,0.5253623127937317
4,4.9265758991241455,3.689648389816284,0.2748299241065979,0.717391312122345,0.7681159377098083
5,5.533876895904541,3.303964614868164,0.29115647077560425,0.7173911929130554,0.6630434989929199
6,5.653120994567871,2.833132743835449,0.30204081535339355,0.7463768124580383,0.695652186870575
7,5.782026052474976,4.43399715423584,0.31292515993118286,0.8115941882133484,0.739130437374115
8,5.88078498840332,3.496011734008789,0.3183673322200775,0.6413043737411499,0.6521739363670349
9,6.110200881958008,4.5207414627075195,0.3224489688873291,0.717391312122345,0.6739130616188049
10,6.191290855407715,3.979151964187622,0.3265306055545807,0.83695

### Validation result

In [11]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)

avg_epoch_time, time, val_loss, val_accuracy, val_precision, val_recall
3.739111361503601,186.95556807518005,7.5587592124938965,0.3592964708805084,0.6384614706039429,0.692307710647583
